In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()


In [3]:
df_a = pd.read_json('../../tmp/erx-arena/results-a.jsonl', lines=True)
df_a.head()

,text,triples,predicted_triples,exact.precision,exact.recall,exact.f1,fuzzy.precision,fuzzy.recall,fuzzy.f1
0,"Caterpillar Inc. is located in Peoria, Illinois.","[Caterpillar Inc. | location | Peoria, Illinois]","[Caterpillar Inc. | is located in | Peoria, Il...",0.0,0.0,0.0,1.000000,1.000000,1.000000
1,Dale Tallon is the general manager of the Flor...,[Florida Panthers | general manager | Dale Tal...,[Dale Tallon | general manager of | Florida Pa...,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,"Aaron Turner played with the band Twilight, is...",[Aaron Turner | associated band/associated mus...,[Aaron Turner | played with | the band Twiligh...,0.0,0.0,0.0,0.333333,0.320000,0.326531
3,"From Spain, Ajoblanco (alternatively known as ...","[Ajoblanco | country | Spain, Ajoblanco | alte...","[From Spain | predicate | Ajoblanco, Ajoblanco...",0.0,0.0,0.0,0.142857,0.333333,0.200000
4,Alfredo Zitarrosa performs Zamba music and sta...,"[Alfredo Zitarrosa | genre | Zamba (artform), ...","[Alfredo Zitarrosa | performs | Zamba music, A...",0.0,0.0,0.0,0.250000,0.235294,0.242424


In [4]:
df_b = pd.read_json('../../tmp/erx-arena/results-a.jsonl', lines=True)
df_b['predicted_triples'] = df_b['triples']
df_b.head()

,text,triples,predicted_triples,exact.precision,exact.recall,exact.f1,fuzzy.precision,fuzzy.recall,fuzzy.f1
0,"Caterpillar Inc. is located in Peoria, Illinois.","[Caterpillar Inc. | location | Peoria, Illinois]","[Caterpillar Inc. | location | Peoria, Illinois]",0.0,0.0,0.0,1.000000,1.000000,1.000000
1,Dale Tallon is the general manager of the Flor...,[Florida Panthers | general manager | Dale Tal...,[Florida Panthers | general manager | Dale Tal...,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,"Aaron Turner played with the band Twilight, is...",[Aaron Turner | associated band/associated mus...,[Aaron Turner | associated band/associated mus...,0.0,0.0,0.0,0.333333,0.320000,0.326531
3,"From Spain, Ajoblanco (alternatively known as ...","[Ajoblanco | country | Spain, Ajoblanco | alte...","[Ajoblanco | country | Spain, Ajoblanco | alte...",0.0,0.0,0.0,0.142857,0.333333,0.200000
4,Alfredo Zitarrosa performs Zamba music and sta...,"[Alfredo Zitarrosa | genre | Zamba (artform), ...","[Alfredo Zitarrosa | genre | Zamba (artform), ...",0.0,0.0,0.0,0.250000,0.235294,0.242424


In [5]:
import re
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """
Compare the triples in the form of `subject | relation | object` extracted by different joint entity relation extraction models from the text given below. 
- The triple set extracted by the model is good if it is complete, correct, consistent with the text and does not contain duplicate triples. 
- Priority: Completeness > Correctness > Consistency > No duplicates

# Text
{text}

# Model A
{triples_a}

# Model B
{triples_b}

# Output
Explanation: [A concise explanation of the comparison]
Result: [A or B or Draw]
"""

def compare_triples(text, triples_a, triples_b):
    response = client.chat.completions.create(
        model="deepseek-r1-llama-70b",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT.format(text=text, triples_a=triples_a, triples_b=triples_b)}
        ]
    )
    return response.choices[0].message.content


def parse_result(comparison):
    result = re.search(r'Result: (.*)', comparison)
    if result:
        return result.group(1)
    else:
        return None

In [6]:
# test compare_triples
text = "Claude is a large language model developed by Anthropic."
triples_a = "Claude | is a | large language model\nClaude | is developed by | Anthropic"
triples_b = "Claude | is a | large language model\nClaude | is developed by | Anthropic\nAnthropic | focus on | AI safety"
comparison = compare_triples(text, triples_a, triples_b)
print(comparison)
print('-'*100)
print(parse_result(comparison))


Alright, I'm trying to figure out how to compare the triples extracted by Model A and Model B from the given text. The text is "Claude is a large language model developed by Anthropic." 

First, I need to understand what the task is. The user wants me to evaluate which model performs better based on certain criteria: completeness, correctness, consistency, and no duplicates, in that order of priority.

Model A extracted two triples: Claude | is a | large language model and Claude | is developed by | Anthropic. Model B has the same two triples but adds a third one: Anthropic | focus on | AI safety.

I should start by checking the text to see what information is present. The text mentions Claude is a large language model and that it's developed by Anthropic. There's no mention of Anthropic focusing on AI safety, so that's extra information from Model B that isn't supported by the text.

Now, evaluating based on the criteria:

1. **Completeness**: Model A captures both relations explicitl

In [7]:
comp_df = pd.merge(df_a, df_b, on='text', how='inner', suffixes=['_A', '_B'])[['text', 'predicted_triples_A', 'predicted_triples_B']].head(2)
comp_df


,text,predicted_triples_A,predicted_triples_B
0,"Caterpillar Inc. is located in Peoria, Illinois.","[Caterpillar Inc. | is located in | Peoria, Il...","[Caterpillar Inc. | location | Peoria, Illinois]"
1,Dale Tallon is the general manager of the Flor...,[Dale Tallon | general manager of | Florida Pa...,[Florida Panthers | general manager | Dale Tal...


In [8]:
comp_df['comparison'] = comp_df.progress_apply(lambda row: compare_triples(row['text'], row['predicted_triples_A'], row['predicted_triples_B']), axis=1)
comp_df['result'] = comp_df['comparison'].apply(parse_result)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [06:32<00:00, 196.03s/it]


In [9]:
comp_df

,text,predicted_triples_A,predicted_triples_B,comparison,result
0,"Caterpillar Inc. is located in Peoria, Illinois.","[Caterpillar Inc. | is located in | Peoria, Il...","[Caterpillar Inc. | location | Peoria, Illinois]","Alright, I need to figure out which model, A o...",A
1,Dale Tallon is the general manager of the Flor...,[Dale Tallon | general manager of | Florida Pa...,[Florida Panthers | general manager | Dale Tal...,"Okay, let me try to figure this out. So, I hav...",B
